In [1]:
max_length=256 #sms(메시지)최대 길이

# 1. 데이터 불러오기

In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('sms.tsv',sep='\t',)
print(df.columns)
print(df.shape)

Index(['label', 'sms'], dtype='object')
(5572, 2)


In [7]:
df.head()

,label,sms
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# 클래스 파악
classes=sorted(set (df['label']))
class_to_idx={}

for i, c in enumerate(classes): #모든 클래스에 대해
    class_to_idx.update({c: i})
    
nclass=len(classes)

print("# of classes: %d" %nclass)
print(classes)
print(class_to_idx)

# of classes: 2
['ham', 'spam']
{'ham': 0, 'spam': 1}


# 2. 새로운 DataFrame

## 1)'label, sms'만 남기기 sms에는 안해도 됨(이미 저 두개만 있음)
## 2) 최대 텍스트 길이만큼 자르기 #pandas.Series.str.slice

In [8]:
# 성별 가사만 남기고싶으면(kpop에서) 속성만 바꿔주면 된다!

new_df=pd.DataFrame({'label':df['label'],
                    'sms':df['sms'].str.slice( #최대 텍스트만큼 자르기
                        start=0,stop=max_length)
                    })

## 3) 중복 제거

In [9]:
len(new_df)

5572

In [11]:
new_df=pd.DataFrame(new_df.drop_duplicates())

In [17]:
len(new_df)

5169

## 4) 셔플

In [18]:
df_shuffled=new_df.sample(frac=1).reset_index(drop=True)
df_shuffled.head()

,label,sms
0,ham,G.W.R
1,ham,But i haf enuff space got like 4 mb...
2,ham,Oops my phone died and I didn't even know. Yea...
3,ham,Nope wif my sis lor... Aft bathing my dog then...
4,spam,TheMob> Check out our newest selection of cont...


## 5) train, test 나누기

In [20]:
# train: test=9:1
train_ratio=0.9

#train dataset
s,e=0, int(df_shuffled.shape[0]*train_ratio) # number of rows
df_train=pd.DataFrame({'label':df_shuffled['label'][s:e],
                      'sms':df_shuffled['sms'][s:e]})
print("Index for test: %d~%d" %(s,e))

#testdataset
s,e=e,e+int(df_shuffled.shape[0]*(1.0-train_ratio))
print("Index for test: %d~%d" %(s,e))
df_test=pd.DataFrame({'label':df_shuffled['label'][s:e],
                      'sms':df_shuffled['sms'][s:e]})

Index for test: 0~4652
Index for test: 4652~5168


In [21]:
# column 수 확인
print(df_train.shape)
print(df_test.shape)

(4652, 2)
(516, 2)


## 6) 저장

In [22]:
df_train.to_csv('./sms.maxlen.uniq.shuf.train.tsv',
               header=False, index=False, sep='\t')
df_test.to_csv('./sms.maxlen.uniq.shuf.test.tsv',
               header=False, index=False, sep='\t')

##

#